In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("dataset/trade_view_registration_case_for_prince.parquet")
# df["countryName"].unique()
df.head()

,countryName,city
0,INDIA,MAHARASTRA STAYTE
1,ARMENIA,d
2,BURKINA FASO,Banfora
3,PHILIPPINES,"Sta.Rosa, Laguna"
4,CHINA,Wuxi/jiangsu


In [3]:
master = pd.read_parquet("https://github.com/storemesh/masterdata/raw/django-app/master-area_20230810.parquet")
master.head()

,global_code,global_name,region_code,region_name,sub_region_code,sub_region_name,intermediate_region_code,intermediate_region_name,country_or_area,m49_code,iso_alpha2_code,iso_alpha3_code,year,id
0,001,World,142,Asia,034,Southern Asia,000,,Afghanistan,004,AF,AFG,2023,2023AFG
1,001,World,150,Europe,039,Southern Europe,000,,Albania,008,AL,ALB,2023,2023ALB
2,001,World,000,,000,,000,,Antarctica,010,AQ,ATA,2023,2023ATA
3,001,World,002,Africa,015,Northern Africa,000,,Algeria,012,DZ,DZA,2023,2023DZA
4,001,World,009,Oceania,061,Polynesia,000,,American Samoa,016,AS,ASM,2023,2023ASM


In [4]:
master.dropna(subset='iso_alpha2_code',inplace=True)
master_dict = master[["country_or_area","iso_alpha2_code"]].to_dict("records")
master_dict = {elm.get("country_or_area").lower():elm.get("iso_alpha2_code") for elm in master_dict}
len(master_dict)

248

In [5]:
from data_lib import Monad as mo

In [6]:
column_name = 'countryName'
df[column_name+'_result'] = df[column_name].apply(lambda x: mo.Monad(x)| mo.country_name_01 | mo.country_name_02 )
df

,countryName,city,countryName_result
0,INDIA,MAHARASTRA STAYTE,passed(IN)['Passed:country_name_01()']
1,ARMENIA,d,passed(AM)['Passed:country_name_01()']
2,BURKINA FASO,Banfora,passed(BF)['Passed:country_name_01()']
3,PHILIPPINES,"Sta.Rosa, Laguna",passed(PH)['Passed:country_name_01()']
4,CHINA,Wuxi/jiangsu,passed(CN)['Passed:country_name_01()']
...,...,...,...
59408,INDIA,Lucknow,passed(IN)['Passed:country_name_01()']
59409,INDIA,Shahjahanpur,passed(IN)['Passed:country_name_01()']
59410,THAILAND,Province,passed(TH)['Passed:country_name_01()']
59411,ISRAEL,,passed(IL)['Passed:country_name_01()']


In [74]:
column_name = 'countryName'
df[column_name+'_result'] = df[column_name].apply(lambda x: Monad(x)|country_name_01 | country_name_02 )
df

,countryName,city,countryName_result
0,INDIA,MAHARASTRA STAYTE,passed(IN)['Passed:country_name_01()']
1,ARMENIA,d,passed(AM)['Passed:country_name_01()']
2,BURKINA FASO,Banfora,passed(BF)['Passed:country_name_01()']
3,PHILIPPINES,"Sta.Rosa, Laguna",passed(PH)['Passed:country_name_01()']
4,CHINA,Wuxi/jiangsu,passed(CN)['Passed:country_name_01()']
...,...,...,...
59408,INDIA,Lucknow,passed(IN)['Passed:country_name_01()']
59409,INDIA,Shahjahanpur,passed(IN)['Passed:country_name_01()']
59410,THAILAND,Province,passed(TH)['Passed:country_name_01()']
59411,ISRAEL,,passed(IL)['Passed:country_name_01()']


In [20]:
class Monad:
    def __init__(self, value):
        self.value = value
        self.status = 'dirty'  # Starting with 'dirty'
        self.message = []
        self.dtype = object
    def __or__(self, func):
        if self.status == 'dirty': # Only process if 'dirty'
            try:
                x = func(self.value)
                self.value = x
                self.status = 'passed'
                self.message.append(f'Passed:{func.__name__}()')
                return self 
            except Exception as e:
                self.message.append(f'Failed:{func.__name__}():{str(e)}')
        return self

    def __repr__(self):
        return f'{self.status}({self.value}){self.message}'

    def __str__(self):
        return self.__repr__()

In [72]:
def country_code(x):
    valid_countries = ['TH', 'US', 'UK']
    if x not in valid_countries:
        raise Exception("Invalid code")
    else:
        return x
        
def country_name(x):
    valid_countries ={'Thailand':'TH','United State':'USA', 'United Kingdom':'UK'}
    return valid_countries[x]

def country_name_thai(x):
    valid_countries ={'ไทย':'TH','สหรัฐอเมริกา':'USA', 'อังกฤษ':'UK'}
    return valid_countries[x]

def province_name(x):
    valid_province = province_set
    if x not in valid_province:
        raise Exception("Invalid code")
    else:
        return x

def clean_datetime(x):
    return pd.to_datetime(x,errors="raise")

def country_name_01(x):
    valid_country = master_dict
    return valid_country[x.lower()]

def country_name_02(x):
    master_dict = {"vietnam":"VN",
                    "korea" : "KR",
                   "taiwan": "TW",
                   "united kingdom": "GB",
                   "hong kong" : "HK",
                   "russia" : "RU",
                   "laos" : "LA",
                   "netherlands" : "NL",
                   "turkey" : "TR",
                   "iran" : "IR",
                   "republic of south africa" : "ZA"
                   
                  }
    
    valid_country = master_dict
    return valid_country[x.lower()]

In [75]:
df_result=pd.DataFrame({
    'input':df[column_name],
    'value':df[column_name+'_result'].apply(lambda x: x.value),
    'status':df[column_name+'_result'].apply(lambda x: x.status),
    'message':df[column_name+'_result'].apply(lambda x: x.message),
    
})
df_result['column']=column_name
df2_result = df_result.explode('message').reset_index(names=['row'])
df2_result

,row,input,value,status,message,column
0,0,INDIA,IN,passed,Passed:country_name_01(),countryName
1,1,ARMENIA,AM,passed,Passed:country_name_01(),countryName
2,2,BURKINA FASO,BF,passed,Passed:country_name_01(),countryName
3,3,PHILIPPINES,PH,passed,Passed:country_name_01(),countryName
4,4,CHINA,CN,passed,Passed:country_name_01(),countryName
...,...,...,...,...,...,...
67247,59409,INDIA,IN,passed,Passed:country_name_01(),countryName
67248,59410,THAILAND,TH,passed,Passed:country_name_01(),countryName
67249,59411,ISRAEL,IL,passed,Passed:country_name_01(),countryName
67250,59412,NETHERLANDS,NL,passed,Failed:country_name_01():'netherlands',countryName


In [76]:
df_fail = df2_result[df2_result["status"]=="dirty"]
df_fail

,row,input,value,status,message,column
107,90,PALESTINE,PALESTINE,dirty,Failed:country_name_01():'palestine',countryName
108,90,PALESTINE,PALESTINE,dirty,Failed:country_name_02():'palestine',countryName
191,160,GAMBIA,GAMBIA,dirty,Failed:country_name_01():'gambia ',countryName
192,160,GAMBIA,GAMBIA,dirty,Failed:country_name_02():'gambia ',countryName
240,197,PALESTINE,PALESTINE,dirty,Failed:country_name_01():'palestine',countryName
...,...,...,...,...,...,...
67064,59236,GUINEA ( GUINEE ),GUINEA ( GUINEE ),dirty,Failed:country_name_02():'guinea ( guinee )',countryName
67108,59279,GUINEA ( GUINEE ),GUINEA ( GUINEE ),dirty,Failed:country_name_01():'guinea ( guinee )',countryName
67109,59279,GUINEA ( GUINEE ),GUINEA ( GUINEE ),dirty,Failed:country_name_02():'guinea ( guinee )',countryName
67117,59287,GUINEA ( GUINEE ),GUINEA ( GUINEE ),dirty,Failed:country_name_01():'guinea ( guinee )',countryName


In [77]:
df_fail.groupby("input").size().sort_values(ascending=False)

input
GUINEA  ( GUINEE )             302
BRUNEI                         182
CZECH REPUBLIC                 158
TANZANIA                       122
REUNION                        100
PALESTINE                       72
SYRIA                           66
MACAO                           64
TRINIDAD                        60
GAMBIA                          50
VIRGIN ISLANDS (USA)            38
BOLIVIA                         26
COTE DIVOIRE                    24
EAST TIMOR                      20
ARUBA, BONAIRE AND CURACAO      16
UNITED STATES MISCELLANEO       16
IVORY COAST                     14
BRITISH INDIAN OCEAN TERRI.     14
VENEZUELA                       14
MOLDOVA                         12
SWAZILAND                       12
ST.CHRISTOPHER AND NEVIS        12
HAWAII                          10
TAHITI                          10
SAINT BARTHELEMY                 8
SCOTLAND                         6
UNITED STATES VIRGIN ISL.        6
ZAIRE                            6
SAIPAN        

In [64]:
#df_fail[df_fail["value"]=="VIETNAM"].iloc[0].message

In [1]:
master[master["country_or_area"].str.lower().str.contains("tai wan")]

NameError: name 'master' is not defined